In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime
from datetime import datetime, timedelta
from tqdm import tqdm_notebook as tqdm
import calendar

from multiprocessing import Pool, TimeoutError, Lock, Value, current_process, cpu_count
from functools import partial

import Feature_function as func

In [2]:
data = pd.read_csv("D:\\Strategic_Trading\\PriceForecast\\data\\2330\\Daily.csv", converters = {"ts": str})
data["ts"] = pd.to_datetime(data["ts"])
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662
...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577


In [3]:
data["Month"] = data["ts"].dt.month
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index,Month
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634,7
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769,7
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296,7
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809,7
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222,6
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780,6
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877,6
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577,6


In [4]:
data["Weekday"] = data["ts"].dt.weekday + 1
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index,Month,Weekday
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634,7,1
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769,7,2
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296,7,3
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809,7,4
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222,6,1
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780,6,2
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877,6,3
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577,6,4


In [5]:
year_list = data["ts"].dt.year.unique().tolist()
month_list = data["ts"].dt.month.unique().tolist()
month_dict = {}
for y in year_list:
    month_dict[y] = {}
    for m in month_list:
        d = data[(data["ts"].dt.year == y) & (data["ts"].dt.month == m)]["ts"].dt.date.unique().tolist()
        month_dict[y][m] = {}
        for i, date in enumerate(d):
            month_dict[y][m][date] = i

month_dict

{2013: {7: {datetime.date(2013, 7, 1): 0,
   datetime.date(2013, 7, 2): 1,
   datetime.date(2013, 7, 3): 2,
   datetime.date(2013, 7, 4): 3,
   datetime.date(2013, 7, 5): 4,
   datetime.date(2013, 7, 8): 5,
   datetime.date(2013, 7, 9): 6,
   datetime.date(2013, 7, 10): 7,
   datetime.date(2013, 7, 11): 8,
   datetime.date(2013, 7, 12): 9,
   datetime.date(2013, 7, 15): 10,
   datetime.date(2013, 7, 16): 11,
   datetime.date(2013, 7, 17): 12,
   datetime.date(2013, 7, 18): 13,
   datetime.date(2013, 7, 19): 14,
   datetime.date(2013, 7, 22): 15,
   datetime.date(2013, 7, 23): 16,
   datetime.date(2013, 7, 24): 17,
   datetime.date(2013, 7, 25): 18,
   datetime.date(2013, 7, 26): 19,
   datetime.date(2013, 7, 29): 20,
   datetime.date(2013, 7, 30): 21,
   datetime.date(2013, 7, 31): 22},
  8: {datetime.date(2013, 8, 1): 0,
   datetime.date(2013, 8, 2): 1,
   datetime.date(2013, 8, 5): 2,
   datetime.date(2013, 8, 6): 3,
   datetime.date(2013, 8, 7): 4,
   datetime.date(2013, 8, 8): 5,
 

In [6]:
data["Intramonth"] = data.apply(func.IntraMonth, month_dict=month_dict, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index,Month,Weekday,Intramonth
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634,7,1,0.000000
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769,7,2,0.045455
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296,7,3,0.090909
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809,7,4,0.136364
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662,7,5,0.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222,6,1,0.789474
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780,6,2,0.842105
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877,6,3,0.894737
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577,6,4,0.947368


In [7]:
data["IntramonthPeriod"] = data.apply(func.MonthPeriod, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index,Month,Weekday,Intramonth,IntramonthPeriod
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634,7,1,0.000000,1
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769,7,2,0.045455,1
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296,7,3,0.090909,1
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809,7,4,0.136364,1
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662,7,5,0.181818,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222,6,1,0.789474,4
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780,6,2,0.842105,5
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877,6,3,0.894737,5
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577,6,4,0.947368,5


In [8]:
data["WeekNum"] = data.apply(func.WeekTime, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634,7,1,0.000000,1,0.0
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769,7,2,0.045455,1,0.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296,7,3,0.090909,1,0.0
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809,7,4,0.136364,1,0.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662,7,5,0.181818,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222,6,1,0.789474,4,1.0
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780,6,2,0.842105,5,1.0
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877,6,3,0.894737,5,1.0
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577,6,4,0.947368,5,1.0


In [9]:
data["MA20"] = data.close.rolling(20).mean()
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,close_index,vol_index,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum,MA20
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,8036.00,3744634,7,1,0.000000,1,0.0,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,8015.86,3539769,7,2,0.045455,1,0.0,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,7911.42,3902296,7,3,0.090909,1,0.0,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,7893.72,3414809,7,4,0.136364,1,0.0,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,8001.82,3604662,7,5,0.181818,1,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,10786.46,5515222,6,1,0.789474,4,1.0,226.675
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,10742.17,6515780,6,2,0.842105,5,1.0,226.050
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,10701.03,5985877,6,3,0.894737,5,1.0,225.450
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,10654.28,5790577,6,4,0.947368,5,1.0,225.000


In [10]:
data["close_lag"] = data["close"].shift(1)
data["high_lag"] = data["high"].shift(1)
data["low_lag"] = data["low"].shift(1)
data["return"] = data["close"] - data["close_lag"]
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum,MA20,close_lag,high_lag,low_lag,return
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,7,1,0.000000,1,0.0,NaN,NaN,NaN,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,7,2,0.045455,1,0.0,NaN,108.0,109.5,108.0,2.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,7,3,0.090909,1,0.0,NaN,110.0,110.0,108.5,-3.0
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,7,4,0.136364,1,0.0,NaN,107.0,107.0,105.5,0.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,7,5,0.181818,1,0.0,NaN,107.0,107.0,106.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,6,1,0.789474,4,1.0,226.675,227.5,227.5,224.0,-9.5
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,6,2,0.842105,5,1.0,226.050,218.0,220.5,218.0,-3.5
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,6,3,0.894737,5,1.0,225.450,214.5,218.0,212.5,-1.5
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,6,4,0.947368,5,1.0,225.000,213.0,216.0,212.5,-1.0


In [11]:
data["TR"] = data.apply(func.TR, axis = 1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,Weekday,Intramonth,IntramonthPeriod,WeekNum,MA20,close_lag,high_lag,low_lag,return,TR
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,1,0.000000,1,0.0,NaN,NaN,NaN,NaN,NaN,1.5
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,2,0.045455,1,0.0,NaN,108.0,109.5,108.0,2.0,2.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,3,0.090909,1,0.0,NaN,110.0,110.0,108.5,-3.0,4.5
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,4,0.136364,1,0.0,NaN,107.0,107.0,105.5,0.0,1.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,5,0.181818,1,0.0,NaN,107.0,107.0,106.0,2.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,1,0.789474,4,1.0,226.675,227.5,227.5,224.0,-9.5,9.5
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,2,0.842105,5,1.0,226.050,218.0,220.5,218.0,-3.5,5.5
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,3,0.894737,5,1.0,225.450,214.5,218.0,212.5,-1.5,3.5
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,4,0.947368,5,1.0,225.000,213.0,216.0,212.5,-1.0,4.5


In [12]:
data['SATR'] = data.TR.rolling(20).mean()
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,Intramonth,IntramonthPeriod,WeekNum,MA20,close_lag,high_lag,low_lag,return,TR,SATR
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,0.000000,1,0.0,NaN,NaN,NaN,NaN,NaN,1.5,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,0.045455,1,0.0,NaN,108.0,109.5,108.0,2.0,2.0,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.090909,1,0.0,NaN,110.0,110.0,108.5,-3.0,4.5,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.136364,1,0.0,NaN,107.0,107.0,105.5,0.0,1.0,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,0.181818,1,0.0,NaN,107.0,107.0,106.0,2.0,2.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.789474,4,1.0,226.675,227.5,227.5,224.0,-9.5,9.5,5.400
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.842105,5,1.0,226.050,218.0,220.5,218.0,-3.5,5.5,5.500
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.894737,5,1.0,225.450,214.5,218.0,212.5,-1.5,3.5,5.500
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.947368,5,1.0,225.000,213.0,216.0,212.5,-1.0,4.5,5.500


In [13]:
data["EATR"] = np.nan
for i in tqdm(range(len(data))):
    if i < 19:
        continue
    elif i == 19 :
        data["EATR"][i] = data["SATR"][i]
    else:
        data["EATR"][i] = func.EATR(i, data, 20)

data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,IntramonthPeriod,WeekNum,MA20,close_lag,high_lag,low_lag,return,TR,SATR,EATR
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,1,0.0,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,1,0.0,NaN,108.0,109.5,108.0,2.0,2.0,NaN,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,1,0.0,NaN,110.0,110.0,108.5,-3.0,4.5,NaN,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,1,0.0,NaN,107.0,107.0,105.5,0.0,1.0,NaN,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,1,0.0,NaN,107.0,107.0,106.0,2.0,2.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,4,1.0,226.675,227.5,227.5,224.0,-9.5,9.5,5.400,5.080768
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,5,1.0,226.050,218.0,220.5,218.0,-3.5,5.5,5.500,5.120695
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,5,1.0,225.450,214.5,218.0,212.5,-1.5,3.5,5.500,4.966343
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,5,1.0,225.000,213.0,216.0,212.5,-1.0,4.5,5.500,4.921929


In [14]:
data["AD"] = np.zeros(len(data))
for i in tqdm(range(len(data))):
        if i == 0:
            data["AD"][i] = func.AD(i, data)
        
        else:
            data["AD"][i] = data.at[i - 1, "AD"] + func.AD(i, data)

data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,WeekNum,MA20,close_lag,high_lag,low_lag,return,TR,SATR,EATR,AD
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,0.0,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,-4.120900e+07
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,0.0,NaN,108.0,109.5,108.0,2.0,2.0,NaN,NaN,1.670000e+05
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,NaN,110.0,110.0,108.5,-3.0,4.5,NaN,NaN,3.916100e+07
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.0,NaN,107.0,107.0,105.5,0.0,1.0,NaN,NaN,6.026400e+07
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,0.0,NaN,107.0,107.0,106.0,2.0,2.5,NaN,NaN,7.487760e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,1.0,226.675,227.5,227.5,224.0,-9.5,9.5,5.400,5.080768,1.191050e+09
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,1.0,226.050,218.0,220.5,218.0,-3.5,5.5,5.500,5.120695,1.183872e+09
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,1.0,225.450,214.5,218.0,212.5,-1.5,3.5,5.500,4.966343,1.167264e+09
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,1.0,225.000,213.0,216.0,212.5,-1.0,4.5,5.500,4.921929,1.164317e+09


In [15]:
data["pos_DM"], data["neg_DM"] = data.apply(func.posDM, axis = 1), data.apply(func.negDM, axis = 1)
data["pos_DI"], data["neg_DI"] = data.apply(func.posDI, axis = 1), data.apply(func.negDI, axis = 1)

In [16]:
data["EMA"] = np.nan
for i in tqdm(range(len(data))):
    if i < 19:
        continue
    elif i == 19 :
        data["EMA"][i] = data["MA20"][i]
    else:
        data["EMA"][i] = func.EMA(i, data, 20)

data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,return,TR,SATR,EATR,AD,pos_DM,neg_DM,pos_DI,neg_DI,EMA
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,1.5,NaN,NaN,-4.120900e+07,NaN,NaN,NaN,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,2.0,2.0,NaN,NaN,1.670000e+05,0.5,0.5,25.0,25.0,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,-3.0,4.5,NaN,NaN,3.916100e+07,0.0,0.0,0.0,0.0,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.0,1.0,NaN,NaN,6.026400e+07,0.0,0.5,0.0,50.0,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,2.0,2.5,NaN,NaN,7.487760e+07,2.5,0.0,100.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,-9.5,9.5,5.400,5.080768,1.191050e+09,0.0,0.0,0.0,0.0,226.783180
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,-3.5,5.5,5.500,5.120695,1.183872e+09,0.0,0.0,0.0,0.0,225.613353
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,-1.5,3.5,5.500,4.966343,1.167264e+09,0.0,0.0,0.0,0.0,224.412081
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,-1.0,4.5,5.500,4.921929,1.164317e+09,0.0,0.0,0.0,0.0,223.229978


In [17]:
data["CMO"] = np.nan
for i in tqdm(range(len(data))):
    if i < 19:
        continue
    else:
        data["CMO"][i] = func.CMO(i, data, 20)
        
data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,TR,SATR,EATR,AD,pos_DM,neg_DM,pos_DI,neg_DI,EMA,CMO
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,1.5,NaN,NaN,-4.120900e+07,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,2.0,NaN,NaN,1.670000e+05,0.5,0.5,25.0,25.0,NaN,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,4.5,NaN,NaN,3.916100e+07,0.0,0.0,0.0,0.0,NaN,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,1.0,NaN,NaN,6.026400e+07,0.0,0.5,0.0,50.0,NaN,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,2.5,NaN,NaN,7.487760e+07,2.5,0.0,100.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,9.5,5.400,5.080768,1.191050e+09,0.0,0.0,0.0,0.0,226.783180,-19.266055
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,5.5,5.500,5.120695,1.183872e+09,0.0,0.0,0.0,0.0,225.613353,-22.123894
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,3.5,5.500,4.966343,1.167264e+09,0.0,0.0,0.0,0.0,224.412081,-21.428571
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,4.5,5.500,4.921929,1.164317e+09,0.0,0.0,0.0,0.0,223.229978,-16.981132


In [18]:
data["M"] = (data["high"] + data["low"] + data["close"]) /3

In [19]:
data["CCI"] = np.nan
for i in tqdm(range(len(data))):
    if i < 19:
        continue
    else:
        data["CCI"][i] = func.CCI(i, data)
        
data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,EATR,AD,pos_DM,neg_DM,pos_DI,neg_DI,EMA,CMO,M,CCI
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,-4.120900e+07,NaN,NaN,NaN,NaN,NaN,NaN,108.500000,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,NaN,1.670000e+05,0.5,0.5,25.0,25.0,NaN,NaN,109.500000,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,NaN,3.916100e+07,0.0,0.0,0.0,0.0,NaN,NaN,106.500000,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,NaN,6.026400e+07,0.0,0.5,0.0,50.0,NaN,NaN,106.666667,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,NaN,7.487760e+07,2.5,0.0,100.0,0.0,NaN,NaN,108.500000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,5.080768,1.191050e+09,0.0,0.0,0.0,0.0,226.783180,-19.266055,218.833333,-223.688120
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,5.120695,1.183872e+09,0.0,0.0,0.0,0.0,225.613353,-22.123894,215.000000,-255.972202
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,4.966343,1.167264e+09,0.0,0.0,0.0,0.0,224.412081,-21.428571,213.833333,-232.304295
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,4.921929,1.164317e+09,0.0,0.0,0.0,0.0,223.229978,-16.981132,212.166667,-231.413652


In [20]:
data["uBBAND"], data["lBBAND"] = data["MA20"] + 2*data.close.rolling(20).std(), data["MA20"] - 2*data.close.rolling(20).std()
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,pos_DM,neg_DM,pos_DI,neg_DI,EMA,CMO,M,CCI,uBBAND,lBBAND
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,NaN,NaN,NaN,NaN,NaN,108.500000,NaN,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,0.5,0.5,25.0,25.0,NaN,NaN,109.500000,NaN,NaN,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,0.0,0.0,0.0,NaN,NaN,106.500000,NaN,NaN,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.0,0.5,0.0,50.0,NaN,NaN,106.666667,NaN,NaN,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,2.5,0.0,100.0,0.0,NaN,NaN,108.500000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.0,0.0,0.0,0.0,226.783180,-19.266055,218.833333,-223.688120,233.397116,219.952884
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.0,0.0,0.0,0.0,225.613353,-22.123894,215.000000,-255.972202,234.694438,217.405562
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.0,0.0,0.0,0.0,224.412081,-21.428571,213.833333,-232.304295,235.882237,215.017763
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.0,0.0,0.0,0.0,223.229978,-16.981132,212.166667,-231.413652,236.911958,213.088042


In [21]:
data["OBV"] = np.zeros(len(data))
for i in tqdm(range(len(data))):
    if i < 19:
        continue
    else:
        data["OBV"][i] = func.OBV(i, data)
        
data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,neg_DM,pos_DI,neg_DI,EMA,CMO,M,CCI,uBBAND,lBBAND,OBV
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,NaN,NaN,NaN,NaN,108.500000,NaN,NaN,NaN,0.0
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,0.5,25.0,25.0,NaN,NaN,109.500000,NaN,NaN,NaN,0.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,0.0,0.0,NaN,NaN,106.500000,NaN,NaN,NaN,0.0
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.5,0.0,50.0,NaN,NaN,106.666667,NaN,NaN,NaN,0.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,0.0,100.0,0.0,NaN,NaN,108.500000,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.0,0.0,0.0,226.783180,-19.266055,218.833333,-223.688120,233.397116,219.952884,207022403.0
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.0,0.0,0.0,225.613353,-22.123894,215.000000,-255.972202,234.694438,217.405562,180702403.0
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.0,0.0,0.0,224.412081,-21.428571,213.833333,-232.304295,235.882237,215.017763,157451403.0
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.0,0.0,0.0,223.229978,-16.981132,212.166667,-231.413652,236.911958,213.088042,130931403.0


In [22]:
data["ROC_20"] = (data["close"] /data["close"].shift(20) - 1) * 100
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,pos_DI,neg_DI,EMA,CMO,M,CCI,uBBAND,lBBAND,OBV,ROC_20
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,NaN,NaN,NaN,108.500000,NaN,NaN,NaN,0.0,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,25.0,25.0,NaN,NaN,109.500000,NaN,NaN,NaN,0.0,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,0.0,NaN,NaN,106.500000,NaN,NaN,NaN,0.0,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.0,50.0,NaN,NaN,106.666667,NaN,NaN,NaN,0.0,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,100.0,0.0,NaN,NaN,108.500000,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.0,0.0,226.783180,-19.266055,218.833333,-223.688120,233.397116,219.952884,207022403.0,-4.595186
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.0,0.0,225.613353,-22.123894,215.000000,-255.972202,234.694438,217.405562,180702403.0,-5.506608
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.0,0.0,224.412081,-21.428571,213.833333,-232.304295,235.882237,215.017763,157451403.0,-5.333333
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.0,0.0,223.229978,-16.981132,212.166667,-231.413652,236.911958,213.088042,130931403.0,-4.072398


In [23]:
data["WR"] = np.nan
for i in tqdm(range(len(data))):
    if i < 19:
        continue
    else:
        data["WR"][i] = func.WR(i, data, 20)
        
data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,neg_DI,EMA,CMO,M,CCI,uBBAND,lBBAND,OBV,ROC_20,WR
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,NaN,NaN,108.500000,NaN,NaN,NaN,0.0,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,25.0,NaN,NaN,109.500000,NaN,NaN,NaN,0.0,NaN,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,NaN,NaN,106.500000,NaN,NaN,NaN,0.0,NaN,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,50.0,NaN,NaN,106.666667,NaN,NaN,NaN,0.0,NaN,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,0.0,NaN,NaN,108.500000,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.0,226.783180,-19.266055,218.833333,-223.688120,233.397116,219.952884,207022403.0,-4.595186,-46.666667
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.0,225.613353,-22.123894,215.000000,-255.972202,234.694438,217.405562,180702403.0,-5.506608,-58.333333
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.0,224.412081,-21.428571,213.833333,-232.304295,235.882237,215.017763,157451403.0,-5.333333,-63.333333
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.0,223.229978,-16.981132,212.166667,-231.413652,236.911958,213.088042,130931403.0,-4.072398,-66.666667


In [24]:
data['YearWeek'] = data.apply(func.YearWeek, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,EMA,CMO,M,CCI,uBBAND,lBBAND,OBV,ROC_20,WR,YearWeek
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,NaN,108.500000,NaN,NaN,NaN,0.0,NaN,NaN,27
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,NaN,NaN,109.500000,NaN,NaN,NaN,0.0,NaN,NaN,27
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,NaN,NaN,106.500000,NaN,NaN,NaN,0.0,NaN,NaN,27
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,NaN,NaN,106.666667,NaN,NaN,NaN,0.0,NaN,NaN,27
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,NaN,NaN,108.500000,NaN,NaN,NaN,0.0,NaN,NaN,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,226.783180,-19.266055,218.833333,-223.688120,233.397116,219.952884,207022403.0,-4.595186,-46.666667,26
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,225.613353,-22.123894,215.000000,-255.972202,234.694438,217.405562,180702403.0,-5.506608,-58.333333,26
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,224.412081,-21.428571,213.833333,-232.304295,235.882237,215.017763,157451403.0,-5.333333,-63.333333,26
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,223.229978,-16.981132,212.166667,-231.413652,236.911958,213.088042,130931403.0,-4.072398,-66.666667,26


In [25]:
year_list = data["ts"].dt.year.unique().tolist()
week_dict = {}

for y in year_list:
    week_dict[y] = {}
    week_list = data[(data['ts'].dt.year == y)]['YearWeek'].unique().tolist()
    for w in week_list:
        week_dict[y][w] = {}
        week = data[(data['ts'].dt.year == y) & (data['YearWeek'] == w)]['Weekday']
        start, end = week.iloc[0], week.iloc[-1]
        week_dict[y][w]['Start'], week_dict[y][w]['End'], week_dict[y][w]['close'] = start, end, data[(data['ts'].dt.year == y) & (data['YearWeek'] == w) & (data['Weekday'] == end)]['close'].iloc[0]

week_dict

{2013: {27: {'Start': 1, 'End': 5, 'close': 109.0},
  28: {'Start': 1, 'End': 5, 'close': 110.0},
  29: {'Start': 1, 'End': 5, 'close': 98.2},
  30: {'Start': 1, 'End': 5, 'close': 102.5},
  31: {'Start': 1, 'End': 5, 'close': 100.5},
  32: {'Start': 1, 'End': 5, 'close': 96.4},
  33: {'Start': 1, 'End': 5, 'close': 96.5},
  34: {'Start': 1, 'End': 5, 'close': 97.0},
  35: {'Start': 1, 'End': 5, 'close': 100.5},
  36: {'Start': 1, 'End': 5, 'close': 102.0},
  37: {'Start': 1, 'End': 6, 'close': 102.5},
  38: {'Start': 1, 'End': 3, 'close': 103.0},
  39: {'Start': 1, 'End': 5, 'close': 103.0},
  40: {'Start': 1, 'End': 5, 'close': 104.0},
  41: {'Start': 1, 'End': 5, 'close': 106.5},
  42: {'Start': 1, 'End': 5, 'close': 110.0},
  43: {'Start': 1, 'End': 5, 'close': 107.0},
  44: {'Start': 1, 'End': 5, 'close': 109.5},
  45: {'Start': 1, 'End': 5, 'close': 106.0},
  46: {'Start': 1, 'End': 5, 'close': 104.0},
  47: {'Start': 1, 'End': 5, 'close': 100.0},
  48: {'Start': 1, 'End': 5, 'cl

In [26]:
data['WeeklyReturn'] = data.apply(func.WeeklyReturn, week_dict=week_dict, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,CMO,M,CCI,uBBAND,lBBAND,OBV,ROC_20,WR,YearWeek,WeeklyReturn
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,108.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,-9999.0
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,NaN,109.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,NaN,106.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,NaN,106.666667,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,NaN,108.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,-19.266055,218.833333,-223.688120,233.397116,219.952884,207022403.0,-4.595186,-46.666667,26,-9999.0
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,-22.123894,215.000000,-255.972202,234.694438,217.405562,180702403.0,-5.506608,-58.333333,26,9999.0
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,-21.428571,213.833333,-232.304295,235.882237,215.017763,157451403.0,-5.333333,-63.333333,26,9999.0
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,-16.981132,212.166667,-231.413652,236.911958,213.088042,130931403.0,-4.072398,-66.666667,26,9999.0


In [27]:
data['y_abs'] = data.apply(func.Weeklylabel, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,M,CCI,uBBAND,lBBAND,OBV,ROC_20,WR,YearWeek,WeeklyReturn,y_abs
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,108.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,-9999.0,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,109.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,106.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,106.666667,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,108.500000,NaN,NaN,NaN,0.0,NaN,NaN,27,3.5,up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,218.833333,-223.688120,233.397116,219.952884,207022403.0,-4.595186,-46.666667,26,-9999.0,NaN
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,215.000000,-255.972202,234.694438,217.405562,180702403.0,-5.506608,-58.333333,26,9999.0,NaN
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,213.833333,-232.304295,235.882237,215.017763,157451403.0,-5.333333,-63.333333,26,9999.0,NaN
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,212.166667,-231.413652,236.911958,213.088042,130931403.0,-4.072398,-66.666667,26,9999.0,NaN


In [28]:
data['y_ATR'] = data.apply(func.Weeklylabel_ATR, axis=1)
data

,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,CCI,uBBAND,lBBAND,OBV,ROC_20,WR,YearWeek,WeeklyReturn,y_abs,y_ATR
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,NaN,NaN,NaN,0.0,NaN,NaN,27,-9999.0,NaN,NaN
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0,NaN,NaN
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0,NaN,NaN
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,NaN,NaN,NaN,0.0,NaN,NaN,27,9999.0,NaN,NaN
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,NaN,NaN,NaN,0.0,NaN,NaN,27,3.5,up,flat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,-223.688120,233.397116,219.952884,207022403.0,-4.595186,-46.666667,26,-9999.0,NaN,NaN
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,-255.972202,234.694438,217.405562,180702403.0,-5.506608,-58.333333,26,9999.0,NaN,NaN
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,-232.304295,235.882237,215.017763,157451403.0,-5.333333,-63.333333,26,9999.0,NaN,NaN
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,-231.413652,236.911958,213.088042,130931403.0,-4.072398,-66.666667,26,9999.0,NaN,NaN


In [29]:
data = func.OneHotMonth(data)
data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
data = func.OneHotWeekday(data)
data

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,ts,open,high,low,close,vol,vwap,open_index,high_index,low_index,...,Sep,Oct,Nov,Dec,Mon,Tue,Wed,Thu,Fri,Sat
0,2013-07-01,109.0,109.5,108.0,108.0,41209000.0,108.643888,8062.21,8063.47,7974.15,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-02,109.0,110.0,108.5,110.0,41376000.0,109.525063,8036.00,8064.12,8012.79,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2013-07-03,107.0,107.0,105.5,107.0,38994000.0,106.501167,8015.86,8015.86,7899.08,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2013-07-04,106.5,107.0,106.0,107.0,21103000.0,106.649244,7911.42,7927.08,7877.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2013-07-05,107.5,109.5,107.0,109.0,24356000.0,108.789066,7893.72,8018.93,7893.72,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2018-06-25,220.0,220.5,218.0,218.0,33685000.0,219.100163,10899.28,10899.28,10786.46,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1225,2018-06-26,215.5,218.0,212.5,214.5,26320000.0,214.483587,10786.46,10786.46,10651.42,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1226,2018-06-27,216.0,216.0,212.5,213.0,23251000.0,213.713668,10742.17,10800.34,10701.03,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1227,2018-06-28,211.0,214.5,210.0,212.0,26520000.0,212.183503,10701.03,10723.84,10633.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [31]:
data.to_csv("D:\\Strategic_Trading\\PriceForecast\\data\\2330\\Daily_Feature.csv", index=False)